In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hf-datasets/wheels/multiprocess-0.70.11.1-py37-none-any.whl
/kaggle/input/hf-datasets/wheels/tqdm-4.49.0-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/huggingface_hub-0.0.9-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/filelock-3.0.12-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/typing_extensions-3.10.0.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/certifi-2021.5.30-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl
/kaggle/input/hf-datasets/wheels/numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
/kaggle/input/hf-datasets/wheels/fsspec-2021.6.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/datasets-1.7.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/python_dateutil-2.8.1-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/idna-2.10-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/six-1.16.0-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/pyparsing-2.4.7-py2.p

In [2]:
train=pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")

In [3]:
model_checkpoint="../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"

In [4]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
train["num_tokens_context"]=train["context"].apply(lambda x: len(tokenizer(x)["input_ids"]))

Token indices sequence length is longer than the specified maximum sequence length for this model (1492 > 512). Running this sequence through the model will result in indexing errors


In [6]:
def convert_answers(r):
    start= r[0]
    text= r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [7]:
train = train.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

In [8]:
df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)


In [9]:
from datasets import Dataset

In [10]:
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
s3fs 2022.5.0 requires fsspec==2022.5.0, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1

In [11]:
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [12]:
valid_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language', 'num_tokens_context', 'answers'],
    num_rows: 64
})

In [13]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"
batch_size=4

In [14]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [15]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [17]:
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [18]:
from transformers import default_data_collator

data_collator = default_data_collator

In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [21]:
pip install wandb --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [22]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 13925
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 435
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
0,No log,0.284637


***** Running Evaluation *****
  Num examples = 816
  Batch size = 4
Saving model checkpoint to chaii-qa/checkpoint-435
Configuration saved in chaii-qa/checkpoint-435/config.json
Model weights saved in chaii-qa/checkpoint-435/pytorch_model.bin
tokenizer config file saved in chaii-qa/checkpoint-435/tokenizer_config.json
Special tokens file saved in chaii-qa/checkpoint-435/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=435, training_loss=0.2638397918350395, metrics={'train_runtime': 1933.6824, 'train_samples_per_second': 7.201, 'train_steps_per_second': 0.225, 'total_flos': 9695700997816320.0, 'train_loss': 0.2638397918350395, 'epoch': 1.0})

In [23]:
trainer.save_model("chaii-bert-trained")

Saving model checkpoint to chaii-bert-trained
Configuration saved in chaii-bert-trained/config.json
Model weights saved in chaii-bert-trained/pytorch_model.bin
tokenizer config file saved in chaii-bert-trained/tokenizer_config.json
Special tokens file saved in chaii-bert-trained/special_tokens_map.json


In [24]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [25]:
validation_features = valid_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
valid_feats_small

  0%|          | 0/816 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 816
})

In [27]:
raw_predictions = trainer.predict(valid_feats_small)

***** Running Prediction *****
  Num examples = 816
  Batch size = 4


In [28]:
max_answer_length = 30
import collections

examples = valid_dataset
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [29]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [30]:
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)

Post-processing 64 example predictions split into 816 features.


  0%|          | 0/64 [00:00<?, ?it/s]

In [31]:
references = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset]


In [32]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [33]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,answer,prediction,jaccard
0,dc64d02cf,प्रतिभा पाटील,प्रतिभा पाटील,1.000000
1,98d8542f6,பிங்க்,பிங்க்,1.000000
2,3ce81e663,मिर्ज़ा असद-उल्लाह बेग ख़ां,मिर्ज़ा असद-उल्लाह बेग ख़ां,1.000000
3,40f4e32da,मीटर प्रति सेकेण्ड2,मीटर प्रति सेकेण्ड2,1.000000
4,bfa4380bd,अगरतला,अगरतला,1.000000
...,...,...,...,...
59,3acaa3b86,स्टेगेरिया,स्टेगेरिया,1.000000
60,7a3e4bf22,கி.மு 3000,கி.மு 6000,0.333333
61,de34ee0a0,माइकोलैव ब्लैक ‍सी शिपयार्ड,यूक्रेन के माइकोलैव ब्लैक ‍सी शिपयार्ड,0.666667
62,518fc3aaf,तेरह,तेरह,1.000000


In [34]:
valid_dataset[3]

{'id': '40f4e32da',
 'context': '[1]\nकिसी वस्तु के वेग परिवर्तन की दर को त्वरण (Acceleration) कहते हैं। इसका मात्रक मीटर प्रति सेकेण्ड2 होता है तथा यह एक सदिश राशि हैं।\na\n→\n(\nt\n)\n=\nd\nv\n→\n(\nt\n)\nd\nt\n≡\nv\n→\n˙\n(\nt\n)\n{\\displaystyle {\\vec {a}}(t)={\\frac {\\mathrm {d} {\\vec {v}}(t)}{\\mathrm {d} t}}\\equiv {\\dot {\\vec {v}}}(t)}\nया,\na\n→\n(\nt\n)\n=\nd\n2\nr\n→\n(\nt\n)\nd\nt\n2\n≡\nr\n→\n¨\n(\nt\n)\n{\\displaystyle {\\vec {a}}(t)={\\frac {\\mathrm {d} ^{2}{\\vec {r}}(t)}{\\mathrm {d} t^{2}}}\\equiv {\\ddot {\\vec {r}}}(t)}\nउदाहरण: माना समय t=० पर कोई कण १० मीटर/सेकेण्ड के वेग से उत्तर दिशा में गति कर रहा है। १० सेकेण्ड बाद उसका वेग बढ़कर ३० मीटर/सेकेण्ड (उत्तर दिशा में) हो जाता है। यह मानते हुए कि इस समयान्तराल में त्वरण का मान नियत है, त्वरण का मान = (३० m/s - १० m/s) / १० सेकेण्ड = २ मीटर प्रति सेकेण्ड2 होगा।\nकिसी वस्तु विशेष द्वारा बदला गया वेग ही त्वरण Acceleration कहलाता है।\nस्पर्शरेखीय तथा अभिकेंद्रीय त्वरण किसी वक्र पथ पर गति करते हुए कण का वेग समय के फ

In [35]:
test=pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")

In [36]:
test_dataset = Dataset.from_pandas(test)

In [37]:
test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats_small

  0%|          | 0/67 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 67
})

In [39]:
test_predictions = trainer.predict(test_feats_small)

***** Running Prediction *****
  Num examples = 67
  Batch size = 4


In [40]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [41]:
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, test_predictions.predictions)

Post-processing 5 example predictions split into 67 features.


  0%|          | 0/5 [00:00<?, ?it/s]

In [42]:
sub=pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")

In [43]:
sub['PredictionString'] = sub['id'].apply(lambda r: final_test_predictions[r])
sub.head()

,id,PredictionString
0,22bff3dec,येलन चीन
1,282758170,20 अप्रैल 2010
2,d60987e0e,१२ मार्च १८२४
3,f99c770dc,13
4,40dec1964,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


In [44]:
sub.to_csv('submission.csv', index=False)